In [1]:
# Suppress TensorFlow warnings and info messages
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

import os
os.environ["OPENCV_LOG_LEVEL"]="SILENT"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Available backend options are: "jax", "torch", "tensorflow".
os.environ["KERAS_BACKEND"] = "tensorflow"

import cv2
import time

import os
import json
import keras
import numpy as np
import matplotlib.pyplot as plt

# from functions import *

In [2]:
@keras.saving.register_keras_serializable()
def scaling(x, scale=1.0):
	return x * scale
# Load the FaceNet512 model from Hugging Face Hub
# model = keras.saving.load_model(
# 	"hf://logasja/FaceNet512",
# 	custom_objects={"scaling": scaling}
# )

# load the facenet128 model
model = keras.saving.load_model("hf://logasja/FaceNet")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
# set camera
def setCamera(index: int = 0, width: int = 1280, height: int = 720,):
    cap = cv2.VideoCapture(index)
    # Set the camera resolution (optional)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    # Set the camera frame rate (optional)
    # cap.set(cv2.CAP_PROP_FPS, fps)
    return cap

In [4]:
# get access to camera and configure it
def getCamera(index: int = 0, max_attempt=5):
    # Try to open external cameras
    try:
        while max_attempt:
            print(f"Opening Camera {index}...")
            cap = setCamera(index)
            if not cap.isOpened():
                print(f"Camera {index} is not Available!")
                index += 1
                max_attempt -= 1
            else:
                print(f"Camera {index} is Available!")
                return cap
    except Exception as e:
        print(f"An error occurred while accessing the camera: {e}")
        exit()
    # Open default camera if external cameras are not available
    if not max_attempt:
        print("Opening Default Camera")
        cap = setCamera(index=0)
        if not cap.isOpened():
            print("Default Camera is not Available!")
            raise SystemExit
        return cap

In [5]:
# face detection
def detectFaces(frame, mode: str):
    # Load the Haar Cascade classifier for face detection
    HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + "haarcascade_frontalface_default.xml"))
    # Detect faces in the image
    faceGray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = HaarCascade.detectMultiScale(
        image=faceGray,
        scaleFactor=1.1,
        minNeighbors=10,
        minSize=(128, 128),
        flags=cv2.CASCADE_SCALE_IMAGE,
    )
    
    # Return the coordinates of the detected faces
    if mode == "single":
        if len(faces) > 0:
            return [faces[0]], 1
        else:
            return [[0, 0, 0, 0]], 0 
    elif mode == "all":
        return faces, len(faces)

In [6]:
# Render a rectangle around the detected faces
def drawRectangle(x, y, width, height, frame, label="Face Detected", distance=0.0):
    # Calculate font size based on the width of the rectangle
    # 0.002 is scale factor that determine font size
    font_scale = width * 0.002
    font_size = round(max(0.4, min(0.8, font_scale)), 2)
    
    # Draw a rectangle around the detected face
    frame = cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 1)
    frame = cv2.rectangle(frame, (x,y-40), (x+width, y), (0, 255, 0), -2)
    
    # Draw a label above the rectangle
    frame = cv2.putText(frame, label + ', ' + str(round(distance, 2)), (x+5, y-15), cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 0, 0), 1, cv2.LINE_AA)
    
    return frame

In [127]:
# save face images
def saveFaceImage(label: str, face: list):
    faces_path = os.path.join("./img/", label)
    
    # check if path exist
    if not os.path.exists(faces_path):
        print(f"Creating folder for new face: {label}")
        os.makedirs(faces_path)
    else:
        # print(f"Folder for {label} already exists.")
        # print("Skipping save function...")
        # return
        pass
    
    # counting the number of files in the directory
    file_counter = len(os.listdir(faces_path)) + 1
    
    # generate filename and join it with face path
    filename = os.path.join(faces_path, f"{label}_{file_counter}.jpg")
    
    # create dummy face
    # face = np.random.randint(0, 256, size=(128, 128), dtype=np.uint8)
    # print(face)
    # cv2.imshow("Gambar Random Dibuat", face)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    
    # save new face image
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        cv2.imwrite(filename, face)
        return f"Face image saved as {filename}"
    
    except Exception as e:
        return f"Gagal menyimpan gambar: {e}"

In [123]:
# generate face signature
def generateFaceSignature(label: str):
    # check signature file
    signatures_path = "./signatures.json"
    if os.path.exists(signatures_path):
        print(f"File {signatures_path} already exists. Updating the file.")
        # Check if file is empty or invalid
        with open(signatures_path, "r+") as f:
            content = f.read().strip()
            if not content:
                f.seek(0)
                json.dump({}, f, indent=4)
                f.truncate()
    else:
        print(f"File {signatures_path} does not exist. Creating a new file.")
        # Create an empty JSON file
        json.dump({}, open(signatures_path, mode="w"), indent=4)
    
    # prepare faces data
    faces_path = os.path.join("./img/", label)
    faces_data = []
    for file in os.listdir(faces_path):
        face_path = os.path.join(faces_path, file)
        face_img = cv2.imread(face_path)
        face_img = cv2.resize(face_img, (160, 160))
        face_img = (face_img - face_img.mean()) / face_img.std()
        faces_data.append(face_img)
    faces_data = np.array(faces_data)   # convert list of face to np.array
    
    # generate signature
    print(f"\nFaces data is ready: {faces_data.shape}")
    print("Generating signature...")
    signature = model.predict(faces_data, batch_size=1)
    
    # combine 50 embedding to robust the signature
    signature_mean = np.mean(signature, axis=0)
    signature_median = np.median(signature, axis=0)
    
    # Convert all numpy arrays to lists for JSON serialization
    signatures = []
    signatures.append(signature.tolist())
    signatures.append(signature_mean.tolist())
    signatures.append(signature_median.tolist())
    
    # Load face signatures from database
    data = json.load(open(signatures_path, mode="r"))    
    # Update the database with the new signature
    data.update({label: signatures})
    # Save the updated database to the file
    json.dump(data, open(signatures_path, mode="w"), indent=4)
    
    # signatures[0] -> 50 raw signatures
    # signatures[1] -> mean signature
    # signatures[2] -> median signature
    
    # return signatures

generateFaceSignature(label="yasir")

File ./signatures.json already exists. Updating the file.

Faces data is ready: (50, 160, 160, 3)
Generating signature...
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


In [ ]:
# Access camera
cap = getCamera(index=1)

# Initialize frame rate calculation
prev_frame_time = 0
new_frame_time = 0


# Allow the camera to warm up
time.sleep(2)

# Check if the video capture has been initialized correctly
if not cap.isOpened():
    print("Error: Could not open video device.")
    raise SystemExit

# Streaming camera
while cap.isOpened():
    key = cv2.waitKey(1) & 0xFF
    
    # Read a frame from the camera
    ret, frame = cap.read()
    
    # Check if the frame was read correctly
    if not ret:
        print("Error: Could not read frame.")
        break
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert the frame to BGR format
    frame = cv2.flip(frame, 1)  # Flip the frame horizontally
    
    # Detect faces in the frame
    faces,faces_count = detectFaces(frame, mode="single")
    
    #  Loop through the detected faces
    for x, y, width, height in faces:
        # Draw rectangles around detected face
        frame = drawRectangle(x, y, width, height, frame)
        
        # Extract the face region from the frame
        face = frame[y:y+height, x:x+width]
        # cv2.imshow("Face", face) # Display the face region
        
        # save face image when pressing enter
        if key == ord('\r'):
            saveFaceImage(label="test", face=face)
    
    # FPS calculation
    new_frame_time = time.time()
    fps = 1 / (new_frame_time - prev_frame_time)
    prev_frame_time = new_frame_time
    
    # Displaying information on the frame
    frame_height = frame.shape[0]
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # Reveerse the frame to BGR format
    cv2.putText(frame, f"Detected Faces: {faces_count}", (10, frame_height - 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA) 
    cv2.putText(frame, f"FPS: {int(fps)}", (10, frame_height - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)     # Display FPS on frame
    cv2.putText(frame, f"Shape: {frame.shape}", (10, frame_height - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)    # Shape of the frame
    
    # Display the frame in a window
    cv2.imshow("Camera Streaming", frame)
    time.sleep(0.01)    # Add a small delay to reduce CPU usage

    # Wait for 1 ms and check if the 'q' key is pressed
    if key == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

Opening Camera 1...
Camera 1 is not Available!
Opening Camera 2...


[ WARN:0@4250.939] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@4251.134] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@4251.139] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video2): can't open camera by index
[ERROR:0@4251.141] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


Camera 2 is not Available!
Opening Camera 3...
Camera 3 is Available!
Creating folder for new face: test
